In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from pathlib import Path
import pymc as pm
import arviz as az
import multiprocessing as mp
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel
import nevergrad as ng

from tbdynamics.constants import BURN_IN, OPTI_DRAWS
from autumn.infrastructure.remote import springboard
from tbdynamics.calib_utils import get_bcm
import pandas as pd
from estival.utils.sample import SampleTypes


In [ ]:
params = {
    "start_population_size": 2300000.0,
    "seed_time": 1810.0,
    "seed_num": 10.0,
    "seed_duration": 3.0,
    "screening_scaleup_shape": 0.1,
    "screening_inflection_time": 1993.0,
    "screening_end_asymp": 0.65,
}

bcm = get_bcm(params)

In [ ]:
def optimize_ng(sample):
    opt = eng.optimize_model(bcm, budget=2000, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=4)
    rec= opt.minimize(2000)
    return rec.value[1]

In [ ]:
opt_samples = map_parallel(optimize_ng, bcm.sample.sobol(16, SampleTypes.LIST_OF_DICTS), n_workers=4, mode="process")

In [ ]:
lle_samps = esamp.likelihood_extras_for_samples(opt_samples, bcm)
lle_samps

In [ ]:
best_ll_idx = lle_samps.sort_values("logposterior",ascending=False).index

In [ ]:
init_samps= bcm.sample.convert(opt_samples,SampleTypes.SAMPLEITERATOR)[best_ll_idx].convert(SampleTypes.LIST_OF_DICTS)[0:4]
init_samps

In [ ]:
init_samps[0]

In [ ]:
# lhs_samples = bcm.sample.lhs(8, ci=0.67)
# lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
# lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
# start_lhs = lhs_samples[lhs_sorted.index].iloc[0:8]
# def optimize_ng(idx_sample):
#         idx, sample = idx_sample
#         opt = eng.optimize_model(
#             bcm,
#             budget=100,
#             opt_class=ng.optimizers.TwoPointsDE,
#             obj_function=bcm.logposterior,
#             suggested=sample,
#             num_workers=4,
#             ci=0.67
#         )
#         rec = opt.minimize(OPTI_DRAWS)
#         return idx, rec.value[1]

# opt_samples = map_parallel(
#         optimize_ng, start_lhs.iterrows(), n_workers=2, mode="process"
#     )
# opt_samples = bcm.sample.convert(opt_samples)
# init_samps = opt_samples.convert("list_of_dicts")[0]

In [ ]:
# out_path = Path.cwd() / 'runs/r1004'
# n_chains = 4
# n_samples = 200
# with pm.Model() as pm_model:
#         variables = epm.use_model(bcm)
#         idata_raw = pm.sample(
#             step=[pm.DEMetropolisZ(variables)],
#             draws=50000,
#             tune=10000,
#             cores=4,
#             discard_tuned_samples=False,
#             chains=n_chains,
#             progressbar=True,
#             initvals=init_samps[0:4],
#     )
# idata_raw.to_netcdf(str(out_path / "calib_full_out.nc"))

# burnt_idata = idata_raw.sel(draw=np.s_[BURN_IN:])
# idata_extract = az.extract(burnt_idata, num_samples=n_samples)
# bcm.sample.convert(idata_extract).to_hdf5(out_path / "calib_extract_out.h5")

# spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm)
# spaghetti_res.results.to_hdf(str(out_path / "results.hdf"), "spaghetti")

# like_df = esamp.likelihood_extras_for_idata(idata_raw, bcm)
# like_df.to_hdf(str(out_path / "results.hdf"), "likelihood")

In [ ]:
def calibrate(out_path, bcm, draws, tune):
    def optimize_ng_with_idx(item):
        idx, sample = item
        opt = eng.optimize_model(bcm, budget=100, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=4)
        rec= opt.minimize(100)
        return idx, rec.value[1]

    lhs_samples = bcm.sample.lhs(16, ci=0.67)
    lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
    lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
    best8 = lhs_samples[lhs_sorted.index].iloc[0:8]
    opt_samples_idx = map_parallel(optimize_ng_with_idx, best8.iterrows())
    best_opt_samps = bcm.sample.convert(opt_samples_idx)
    lle_samps = esamp.likelihood_extras_for_samples(opt_samples_idx, bcm)
    init_samps = best_opt_samps.convert(SampleTypes.LIST_OF_DICTS)[0]
    n_chains = 4
    n_samples = 10
    with pm.Model() as pm_model:
        variables = epm.use_model(bcm)
        idata_raw = pm.sample(
            step=[pm.DEMetropolisZ(variables)],
            draws=draws,
            tune=tune,
            cores=4,
            discard_tuned_samples=False,
            chains=n_chains,
            progressbar=True,
            initvals=init_samps,
        )
    idata_raw.to_netcdf(str(out_path / "calib_full_out.nc"))

    burnt_idata = idata_raw.sel(draw=np.s_[20:])
    idata_extract = az.extract(burnt_idata, num_samples=n_samples)
    # bcm.sample.convert(idata_extract).to_hdf5(out_path / "calib_extract_out.h5")

    spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm)
    spaghetti_res.results.to_hdf(str(out_path / "results.hdf"), "spaghetti")

    like_df = esamp.likelihood_extras_for_idata(idata_raw, bcm)
    like_df.to_hdf(str(out_path / "results.hdf"), "likelihood")


def run_calibration(bridge: springboard.task.TaskBridge, bcm, draws, tune):
    import multiprocessing as mp
    mp.set_start_method("forkserver")
    idata_raw = calibrate(bridge.out_path, bcm, draws, tune)
    bridge.logger.info("Calibration complete")

In [ ]:
OUT_PATH = Path.cwd() / 'runs/r1004'
draws= 100
tune = 10
calibrate(OUT_PATH,bcm, draws, tune)


In [ ]:
draws = 500000
tune = 50000

commands = [
    'git clone --branch tb-covid https://github.com/longbui/tbdynamics.git',
    'pip install -e ./tbdynamics',
]

mspec = springboard.EC2MachineSpec(8, 2, 'compute')
run_str = f'd{int(draws / 1000)}k-t{int(tune / 1000)}k-b{int(BURN_IN / 1000)}k'
tspec_args = {'bcm': bcm,'draws': draws, 'tune': tune}
tspec = springboard.TaskSpec(run_calibration, tspec_args)
run_path =  springboard.launch.get_autumn_project_run_path('tbdynamics', 's10042_calibration', run_str)
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path, branch=None, extra_commands=commands)


In [ ]:
# # from autumn.infrastructure.remote import springboard
# rts = springboard.task.RemoteTaskStore()
# rts.cd('projects/tbdynamics/s06043_calibration')
# rts.ls()
# mt = rts.get_managed_task('2024-04-05T1445-d500k-t100k-b25k')
# mt.get_log("crash")

In [ ]:
print(runner.get_log())

In [ ]:
# files =runner.get_managed_task()

In [ ]:
# files.download_all()